# Stock Price Forecasting Using AR/MA Models
This notebook demonstrates forecasting stock Close_diff using MA model with auto-selected parameters.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# Load dataset
file_path = "/mnt/data/ML471_S2_Datafile_Practice.csv"
df = pd.read_csv(file_path)

# Parse Date if exists
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

# Target variable
target = 'Close_diff'

# Drop NA
df = df.dropna()

# Train-test split (80-20)
split = int(len(df)*0.8)
train = df[target].iloc[:split]
test = df[target].iloc[split:]

print("Train size:", len(train))
print("Test size:", len(test))


In [ ]:

# Auto-select MA(q) based on AIC
best_aic = float("inf")
best_q = 0

for q in range(1, 8):
    try:
        model = ARIMA(train, order=(0,q,0)).fit()
        if model.aic < best_aic:
            best_aic = model.aic
            best_q = q
    except:
        pass

print("Best MA(q):", best_q)

# Fit best model
ma_model = ARIMA(train, order=(0,best_q,0)).fit()

# Forecast
forecast = ma_model.forecast(steps=len(test))


In [ ]:

# Evaluation Metrics
mae = mean_absolute_error(test, forecast)
rmse = math.sqrt(mean_squared_error(test, forecast))
mape = np.mean(np.abs((test - forecast) / test)) * 100

print("MAE :", mae)
print("RMSE:", rmse)
print("MAPE:", mape)


In [ ]:

# Visualization
plt.figure(figsize=(12,6))
plt.plot(train.index, train, label="Train")
plt.plot(test.index, test, label="Actual", linestyle="--")
plt.plot(test.index, forecast, label="Forecast", linestyle=":")
plt.title("MA Forecast with Auto-selected Parameters")
plt.legend()
plt.show()
